In [1]:
import numpy as np

In [2]:
def DMD(X, Y, tol=1e-12):
    #X - Nxm, Y - Nxm
    m=X.shape[1]; N=X.shape[0]
    U, Sigma, V = np.linalg.svd(X,full_matrices=False) #svd vraca V*
    V=np.conjugate(V.T)
    #print(Sigma)
    k=m
    for i in range(1,min(N,m)):
        if(Sigma[i]<=Sigma[0]*tol):
            k=i
            break
    U = U[:, :k]; V=V[:, :k]; Sigma = np.array(Sigma[:k])
    S_k = ((np.conjugate(U.T)@Y)@V)/Sigma
    Lambda, W = np.linalg.eig(S_k)
    Z = U@W
    return Z, Lambda


In [3]:
def DMD_exact(X, Y, tol=1e-12):
    """X - Nxm, Y - Nxm; !!Exact DMD ne vraća normirane Z[:,i]"""
    m=X.shape[1]; N=X.shape[0]
    U, Sigma, V = np.linalg.svd(X,full_matrices=False) #svd vraca V*
    V=np.conjugate(V.T)
    #print(Sigma)
    k=m
    for i in range(1,min(N,m)):
        if(Sigma[i]<=Sigma[0]*tol):
            k=i
            break
    U = U[:, :k]; V=V[:, :k]; Sigma = np.array(Sigma[:k])
    S_k = ((np.conjugate(U.T)@Y)@V)/Sigma
    Lambda, W = np.linalg.eig(S_k)
    Z = (Y@V/Sigma)@W/Lambda
    return Z, Lambda

In [4]:
def GEDMD(X, Y, tol=1e-12, type: str=None): 
    """
    type = "exact" if you want exact version, or None (/leave empty) if you don't want exact version. 
    Anything else set as type will do the non-exact version.
    """
    m=X.shape[1]; N=X.shape[0]
    D = np.linalg.norm(X, axis=0)
    X = X/D; Y=Y/D
    U, Sigma, V = np.linalg.svd(X,full_matrices=False) 
    V=np.conjugate(V.T)
    k=m
    for i in range(1,min(N,m)):
        if(Sigma[i]<=Sigma[0]*tol):
            k=i
            break
    U = U[:, :k]; V=V[:, :k]; Sigma = np.array(Sigma[:k])
    B_k = Y @ (V/Sigma)
    S_k = np.conjugate(U.T)@B_k
    Lambda, W = np.linalg.eig(S_k)
    Z = U@W
    if(type=="exact"):
        Z = B_k@W
    r = B_k@W - Z*Lambda
    return Z, Lambda, r

In [5]:
def GEDMDQ_multiple_trajectories(X, Y, tol=1e-12, type: str=None):
    """
    type = "exact" if you want exact version, or None (/leave empty) if you don't want exact version. 
    Anything else set as type will do the non-exact version.
    """
    m=X.shape[1];
    Q, R = np.linalg.qr(np.concatenate((X,Y), axis=1))
    Z, L, r = GEDMD(R[:,:m], R[:,m:], tol, type)
    Z = Q@Z
    return Z, L, r


In [64]:
def GEDMDQ(S, tol=1e-12, type: str=None):
    """
    type = "exact" if you want exact version, or None (/leave empty) if you don't want exact version. 
    Anything else set as type will do the non-exact version.
    """
    m=S.shape[1]-1;
    Q, R = np.linalg.qr(S)
    R_x = R[:(m+1),:m]; R_y = R[:(m+1), 1:(m+1)]
    Z, L, r = GEDMD(R_x, R_y, tol, type)
    Z = Q@Z
    return Z, L, r, Q, R

In [65]:
def DMD_added_snap(S, B, Q, R, tol=1e-12, type: str=None):
    m=S.shape[1]-1; N=S.shape[0]; b=B.shape[1]
    if(B.shape[0]!=N):
        return "new data doesn't have the same dimension as the old data"
    R_new= np.conjugate(Q.T)@B
    Q_1, R_1 = np.linalg.qr(B-Q@R_new)
    R_new=np.concatenate((R_new, R_1), axis=0)
    R = np.concatenate((R, np.zeros((b,m+1))), axis=0)
    R = np.concatenate((R, R_new), axis=1)
    Q = np.concatenate((Q, Q_1), axis=1)
    m=m+b
    R_x = R[:(m+1),:m]; R_y = R[:(m+1), 1:(m+1)] 
    Z, L, r = GEDMD(R_x, R_y, tol, type)
    Z=Q@Z
    return Z, L, r, Q, R


In [48]:
def householder_for_dmd(R, l):

    ###OVO JE ZA KVADRATICAN R - sto R koji ulazi nije, mora doci update!!!
    """R - matrix in trapezoidal form; 
    l - the amount of former dmd values discarded (also the number of sub-diagonals below the main one in R)"""
    N = R.shape[1]; N0 = R.shape[0]
    v_all=np.empty((l+1,N-1))
    hh = np.eye(N, N)


    #HH transf od R (trapezoidalna) 
    for i in range(N-l):
        element=R[i:(i+l+1),i]  
        v = element.copy().reshape(-1,1) #ako stavim v=element - pokazuju na istu memoriju; ne radi!!
        v[0] = v[0] + np.sign(element[0])*np.linalg.norm(element)
        R[i:(i+l+1),i:] = R[i:(i+l+1),i:] - 2/(v.T@v)*v@(v.T@R[i:(i+l+1),i:])
        v_all[:,i] = v.reshape(-1)

    #stupci na kraju
    for i in range(l,1,-1):
        element=R[N-i:N,N-i]  
        v = element.copy().reshape(-1,1)
        v[0] = v[0] + np.sign(element[0])*np.linalg.norm(element)
        R[N-i:N,N-i:] = R[N-i:N,N-i:] - 2/(v.T@v)*v@(v.T@R[N-i:N,N-i:])
        v_all[:,N-i] = np.concatenate((v.reshape(-1),np.zeros(l+1-i)), axis=0)

    ##Q - from HH matrices
    hh_current = np.eye(N, N)
    v_current=v_all[:2,-1].reshape(-1,1)
    hh_current[-2:,-2:] = np.eye(2,2)-2/(v_current.T@v_current)*(v_current@v_current.T)

    hh=hh_current #ovdje @ I

    for i in range(N-3,N-l-1,-1):
        v_current=v_all[:N-i,i].reshape(-1,1)
        hh_current = np.eye(N-i,N-i)-2/(v_current.T@v_current)*(v_current@v_current.T)
        hh[i:i+l+1, :] = hh_current@hh[i:i+l+1, :]

    for i in range(N-l-1,-1,-1):
        v_current = v_all[:,i].reshape(-1,1)
        hh_current = np.eye(l+1,l+1)-2/(v_current.T@v_current)*(v_current@v_current.T)
        hh[i:i+l+1, :] = hh_current@hh[i:i+l+1, :]
        #print(hh)


    return hh, R, v_all

In [8]:
def DMD_discarding_snap(l, Q, R, tol=1e-12, type: str=None):
    m=R.shape[1]
    R = 
    hh, R, _ = householder_for_dmd(R, l)
    R_x = R[:(m-l),:(m-l)]; R_y = R[:(m+1), 2:(m+1)]
    Z, L, r = GEDMD(R_x, R_y, tol, type)
    Z=Q@Z
    return Z, L, r, Q, R, hh

SyntaxError: incomplete input (2842192622.py, line 2)

In [9]:
N=300
m=10
x=np.random.rand(N)
A=np.random.rand(N,N)
A = A/np.linalg.norm(A, 2)

In [10]:
X=np.empty((N,m+1))
X[:,0]=x
for i in range(m):
    X[:,i+1]=A@X[:,i]
X=X+np.random.rand(N,m+1)/N

In [62]:
Z, L = DMD(X[:,:m], X[:, 1:], N*np.finfo(float).resolution)

L

array([ 0.8574 +0.j     ,  0.07429+0.04817j,  0.07429-0.04817j,
       -0.06135+0.0922j , -0.06135-0.0922j , -0.03223+0.03605j,
       -0.03223-0.03605j])

In [12]:
#permutirani podaci - druge eigenvalues - kao što bi i očekivali!
print(X[:5,[1,6,8,9]])
temp=X[:,[1,6,8,9]]
X[:,[6,9]] = temp[:,[0,2]]
X[:,[1,8]] = temp[:,[1,3]]
print(X[:5,[1,6,8,9]])

Z_perm, L_perm = DMD(X[:,:m], X[:, 1:], N*np.finfo(float).resolution)

L_perm


[[0.48296877 0.48149964 0.48342686 0.48053751]
 [0.53007302 0.51252136 0.51193012 0.51089179]
 [0.53568701 0.52213735 0.52053685 0.51976197]
 [0.50928412 0.50391888 0.50392397 0.50263338]
 [0.48820717 0.48804566 0.48772004 0.48778274]]
[[0.48149964 0.48296877 0.48053751 0.48342686]
 [0.51252136 0.53007302 0.51089179 0.51193012]
 [0.52213735 0.53568701 0.51976197 0.52053685]
 [0.50391888 0.50928412 0.50263338 0.50392397]
 [0.48804566 0.48820717 0.48778274 0.48772004]]


array([ 0.99907042+0.j        ,  0.59908186+0.58892313j,
        0.59908186-0.58892313j,  0.09752316+0.7572151j ,
        0.09752316-0.7572151j ,  0.00101044+0.j        ,
       -0.74541296+0.32260765j, -0.74541296-0.32260765j,
       -0.41296868+0.68079755j, -0.41296868-0.68079755j])

In [13]:
Ze, Le = DMD_exact(X[:,:m], X[:, 1:], N*np.finfo(float).resolution)
Le
#isto kao L?

array([ 0.99907042+0.j        ,  0.59908186+0.58892313j,
        0.59908186-0.58892313j,  0.09752316+0.7572151j ,
        0.09752316-0.7572151j ,  0.00101044+0.j        ,
       -0.74541296+0.32260765j, -0.74541296-0.32260765j,
       -0.41296868+0.68079755j, -0.41296868-0.68079755j])

In [66]:
Zg, Lg, rg = GEDMD(X[:,:m], X[:,1:], N*np.finfo(float).resolution)
Lg

array([ 0.8574 +0.j     ,  0.07429+0.04817j,  0.07429-0.04817j,
       -0.06135+0.0922j , -0.06135-0.0922j , -0.03223+0.03605j,
       -0.03223-0.03605j])

In [67]:
Z_qr, L_qr, r_qr, Q, R = GEDMDQ(X, N*np.finfo(float).resolution)
print(L_qr)

B = (A@X[:,m]+np.random.rand(N)/N).reshape((-1,1))

Z_added, L_added, r_added, _,_ = DMD_added_snap(X, B, Q, R, N*np.finfo(float).resolution)
print(L_added)

#provjera.. - dobije se isto
_,L_added_direct,_,_,_ = GEDMDQ(np.concatenate((X, B), axis=1), N*np.finfo(float).resolution)
print(L_added_direct)


[ 0.8574 +0.j       0.07429+0.04817j  0.07429-0.04817j -0.06135+0.0922j
 -0.06135-0.0922j  -0.03223+0.03605j -0.03223-0.03605j]
[11885.14188+0.j          0.85739+0.j          0.12699+0.j
     0.09536+0.0979j      0.09536-0.0979j     -0.08255+0.08415j
    -0.08255-0.08415j    -0.1218 +0.j     ]
[11885.14188+0.j          0.85739+0.j          0.12699+0.j
     0.09536+0.0979j      0.09536-0.0979j     -0.08255+0.08415j
    -0.08255-0.08415j    -0.1218 +0.j     ]


In [16]:
np.linalg.eigvals(A)[:10]

array([ 0.99889218+0.j        ,  0.03518798+0.j        ,
        0.03074104+0.01419679j,  0.03074104-0.01419679j,
       -0.01475095+0.03084843j, -0.01475095-0.03084843j,
        0.01122292+0.03097511j,  0.01122292-0.03097511j,
        0.01722588+0.02766489j,  0.01722588-0.02766489j])

In [17]:
#!pip install pydmd

In [18]:
import pydmd
dmd1 = pydmd.DMD(svd_rank=10)
dmd1.fit(X[:,:m], X[:,1:])
dmd1.eigs

array([ 0.99907042+0.j        ,  0.59908186+0.58892313j,
        0.59908186-0.58892313j,  0.09752316+0.7572151j ,
        0.09752316-0.7572151j ,  0.00101044+0.j        ,
       -0.74541296+0.32260765j, -0.74541296-0.32260765j,
       -0.41296868+0.68079755j, -0.41296868-0.68079755j])

In [19]:
for i in range(dmd1.eigs.shape[0]):
    print(np.linalg.norm(dmd1.reconstructed_data[:,i]-X[:,i]))

5.307377401264979e-15
6.11385142022168e-15
2.025335850645148e-14
9.279754505634503e-15
1.2981393845238354e-14
2.4286283471733946e-14
1.8077896089785274e-14
2.4659078555025766e-14
2.742289078819825e-14
3.0534897328111925e-14


In [20]:
dmd1.reconstructed_data.shape

(300, 10)

In [21]:
#usporedi s rezultatima lin regresije...

In [22]:
#procijena s_i - ne ovako!
for i in range(m):
    s=np.zeros((Z.shape[0]))
    for j in range(Z.shape[1]):
        s=s+Z[:,j]*(L[j]**(i))
    print("norma razlike:",np.linalg.norm(s-X[:,i]))
    #print(s,X[:,i], sep="\n")

norma razlike: 10.016455443530974
norma razlike: 7.742227883398989
norma razlike: 7.7923741309410985
norma razlike: 7.733620097090704
norma razlike: 7.657860074757331
norma razlike: 7.690956929821568
norma razlike: 7.674629077743607
norma razlike: 7.603231651478409
norma razlike: 7.593384541152244
norma razlike: 7.614064728997973


In [23]:
#QR za (X Y) kad X i Y imaju "istu jezgru"
N=20; m=7
A = np.random.rand(N,N)
A=A/np.linalg.norm(A)
x = np.ones((N))
X=np.empty((N,m+1))
X[:,0] = x
for i in range(m):
    X[:,i+1]= A @ X[:,i]
#print(X)

Z = np.concatenate((X[:,:m], X[:,1:]), axis=1)
print(Z.shape)

Q, R = np.linalg.qr(Z, mode="reduced")

R_12 = R[:m,m:]
R_22 = R[m:2*m, m:]

Q_1 = Q[:,:m]
Q_2 = Q[:,m:]


print(np.allclose(X[:,1:],Q_1@R_12 + Q_2@R_22))
print(np.allclose(X[:,1:],Q_1@R_12 + Q_2*R_22[:,m-1]))
print(R_22)
print(R.shape)

(20, 14)
True
True
[[-2.49115420e-17 -3.32414159e-18  2.39346322e-19 -2.97233822e-20
  -7.16402113e-22  9.12206535e-23  1.12332284e-07]
 [ 0.00000000e+00 -2.86965239e-17 -4.05206363e-20 -4.52793389e-20
   8.81409224e-21 -4.75823460e-22  5.45047822e-07]
 [ 0.00000000e+00  0.00000000e+00  7.17838529e-19  1.44221062e-20
  -3.22977382e-21 -2.39357798e-22 -2.78134589e-08]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.20583572e-19
  -2.05737289e-22  1.46630486e-22  8.80566342e-08]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -1.85650177e-20  1.51774526e-22  2.73137764e-08]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -8.76165427e-22 -2.36115059e-07]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -5.39037208e-07]]
(14, 14)


In [46]:
"""HH transformacija - proba s trapezoidalnom matricom koja ima 2 sporedne diagonale"""
dim_trapez_m = 6

np.set_printoptions(precision=8, suppress=True)

trapez_m = np.random.rand(dim_trapez_m,dim_trapez_m)
trapez_m = np.triu(trapez_m, -2)
print(trapez_m)
trapez_copy = trapez_m.copy()
v_all=np.empty((3,dim_trapez_m-1))
#hh1 = np.eye(dim_trapez_m, dim_trapez_m)


#HH transf od R (trapezoidalna) 
#### racun od HH matrice ("Q") bi se mogao i ovdje dogoditi - primjena transformacija na I i onda uzmemo transponirano
for i in range(dim_trapez_m-2):
    element=trapez_m[i:(i+3),i]  
    v = element.copy().reshape(-1,1) #ako stavim v=element - pokazuju na istu memoriju; ne radi!!
    v[0] = v[0] + np.sign(element[0])*np.linalg.norm(element)
    trapez_m[i:(i+3),i:] = trapez_m[i:(i+3),i:] - 2/(v.T@v)*v@(v.T@trapez_m[i:(i+3),i:])
    #hh1[i:(i+3),i:] = hh1[i:(i+3),i:] - 2/(v.T@v)*v@(v.T@hh1[i:(i+3),i:])  #- daje krivo iz nekog razloga..
    #print("hh1",hh1,sep="\n")
    v_all[:,i] = v.reshape(-1)

#predzadnji stupac
element=trapez_m[dim_trapez_m-2:dim_trapez_m,dim_trapez_m-2]  
v = element.copy().reshape(-1,1)
v[0] = v[0] + np.sign(element[0])*np.linalg.norm(element)
trapez_m[dim_trapez_m-2:dim_trapez_m,dim_trapez_m-2:] = trapez_m[dim_trapez_m-2:dim_trapez_m,dim_trapez_m-2:] - 2/(v.T@v)*v@(v.T@trapez_m[dim_trapez_m-2:dim_trapez_m,dim_trapez_m-2:])
#hh1[dim_trapez_m-2:dim_trapez_m,dim_trapez_m-2:] = hh1[dim_trapez_m-2:dim_trapez_m,dim_trapez_m-2:] - 2/(v.T@v)*v@(v.T@hh1[dim_trapez_m-2:dim_trapez_m,dim_trapez_m-2:])
v_all[:3,dim_trapez_m-2] = np.concatenate((v.reshape(-1),[0]), axis=0)

#print("hh1_fin",hh1.T, sep="\n")

##Q - from HH matrices
hh_current = np.eye(dim_trapez_m, dim_trapez_m)
v_current=v_all[:2,-1].reshape(-1,1)
hh_current[-2:,-2:] = np.eye(2,2)-2/(v_current.T@v_current)*(v_current@v_current.T)

hh=hh_current #ovdje @ I
#print(hh)

#print(v_all)
for i in range(dim_trapez_m-3,-1,-1):
    v_current = v_all[:,i].reshape(-1,1)
    hh_current = np.eye(3,3)-2/(v_current.T@v_current)*(v_current@v_current.T)
    hh[i:i+3, :] = hh_current@hh[i:i+3, :]
    #print(hh)
print("hh_pravi",hh, sep="\n")

print(trapez_m)
print(np.allclose(hh@trapez_m,trapez_copy))
np.set_printoptions()

[[0.7277402  0.31285009 0.60445674 0.16696111 0.13697844 0.98605776]
 [0.08610975 0.80853874 0.98211818 0.00555537 0.63069454 0.43699647]
 [0.85231584 0.1807425  0.32739857 0.39629574 0.43756928 0.23404151]
 [0.         0.47329275 0.90318252 0.24260367 0.5394951  0.37876998]
 [0.         0.         0.63515159 0.85469207 0.94268736 0.42347423]
 [0.         0.         0.         0.674465   0.80993264 0.78157905]]
hh_pravi
[[-0.64743341 -0.05745596  0.09650832 -0.25205097  0.57374316 -0.41893202]
 [-0.07660746 -0.84508636 -0.22588411  0.12725104 -0.31983959 -0.33233494]
 [-0.75826201  0.13443747 -0.05958139  0.20235464 -0.45757058  0.39127625]
 [ 0.         -0.51425132  0.34484429 -0.1280546   0.34188048  0.69523216]
 [ 0.          0.          0.90398838  0.12089145 -0.30174712 -0.27773899]
 [ 0.          0.          0.          0.92104145  0.38886631 -0.0215787 ]]
[[-1.12403869 -0.40153987 -0.71483697 -0.40901779 -0.46879249 -0.84934871]
 [ 0.         -0.92035302 -1.28515249 -0.08576994 

In [61]:
#test hh fje
trapez_m = np.random.rand(8,8)
trapez_m = np.triu(trapez_m, -4)
trapez_copy=trapez_m.copy()
np.set_printoptions(precision=5, suppress=True)
print(trapez_m)

hh, R = householder_for_dmd(trapez_m, 4)
print(R)
print(np.allclose(hh@R,trapez_copy))

np.set_printoptions()

[[0.04446 0.18314 0.22858 0.90923 0.4084  0.4004  0.30709 0.29643]
 [0.0098  0.5264  0.42379 0.06879 0.45616 0.46564 0.15839 0.82974]
 [0.49822 0.2159  0.76537 0.40558 0.89455 0.84454 0.32758 0.89768]
 [0.63399 0.6591  0.29298 0.58112 0.61438 0.04871 0.02287 0.24053]
 [0.17322 0.66847 0.53796 0.76461 0.93675 0.89142 0.88127 0.27871]
 [0.      0.92795 0.11565 0.43832 0.92822 0.17688 0.53479 0.37237]
 [0.      0.      0.50808 0.98125 0.3182  0.30641 0.67535 0.49937]
 [0.      0.      0.      0.52075 0.93309 0.7399  0.1054  0.09359]]
[[-0.82598 -0.79242 -0.81668 -0.9008  -1.235   -0.76082 -0.41837 -0.81033]
 [-0.      -1.21313 -0.36523 -0.72322 -1.17208 -0.5688  -0.80722 -0.60433]
 [-0.       0.      -0.81488 -0.83255 -0.59838 -0.95242 -0.7821  -0.9095 ]
 [ 0.       0.       0.       1.14752  0.5102   0.28092  0.39239 -0.15572]
 [-0.       0.      -0.       0.      -0.86383 -0.72808 -0.01034 -0.14985]
 [ 0.       0.       0.       0.      -0.       0.37619 -0.00138 -0.21365]
 [ 0.       0